<a href="https://colab.research.google.com/github/yooon27/Embedded-Software-Contest2023/blob/main/w_o_uninstall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import os

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving images_jetson.zip to images_jetson.zip


In [ ]:
!unzip -qq '/content/images_jetson.zip' -d '/content'

In [ ]:
data_dir = '/content/images_jetson'

In [ ]:
# 데이터 전처리
train_datagen = ImageDataGenerator(rescale=1./255)  # 이미지 스케일링
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),  # ResNet50 입력 크기에 맞게 조정
    batch_size=32,
    class_mode='categorical'  # 분류 작업일 경우
)

Found 264 images belonging to 4 classes.


In [ ]:

# ResNet50 모델 로드 (사전 훈련된 가중치 사용)
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모델 아키텍처 정의 (ResNet50의 상위 레이어 추가)
x = Flatten()(resnet.output)
x = Dense(128, activation='relu')(x)
output = Dense(4, activation='softmax')(x)  # num_classes: 분류할 클래스 수
model = Model(inputs=resnet.input, outputs=output)

# 모델 컴파일
model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy',  # 분류 작업일 경우
              metrics=['accuracy'])

# 모델 학습
model.fit(train_generator, epochs=2)  # 적절한 에포크 수로 변경

# 학습된 모델 저장
model.save('path/to/save/model')

94765736/94765736 [==============================] - 0s 0us/step


Epoch 1/2
9/9 [==============================] - 236s 23s/step - loss: 3.0550 - accuracy: 0.8144
Epoch 2/2
9/9 [==============================] - 204s 22s/step - loss: 1.7489 - accuracy: 0.9811


In [ ]:

from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# 학습된 모델 로드
model = tf.keras.models.load_model('path/to/save/model')

In [ ]:
# 입력 이미지셋 저장
from google.colab import files
uploaded = files.upload()

Saving testset.zip to testset.zip


In [ ]:
!unzip -qq '/content/testset.zip' -d '/content'

In [ ]:

while True:
    # 사용자로부터 숫자 입력받기
    image_number = input("Enter the image number ('q' to quit): ")

    if image_number.lower() == 'q':
        break

    image_path = f'/content/testset/left_{image_number.zfill(5)}.jpg'

    # 입력 이미지 전처리
    input_image = load_img(image_path, target_size=(224, 224))
    input_image = img_to_array(input_image)
    input_image = np.expand_dims(input_image, axis=0)
    input_image = input_image / 255.0  # 이미지 스케일링

    # 예측 수행
    predictions = model.predict(input_image)
    predicted_class_index = np.argmax(predictions[0])
    predicted_class = ['left', 'right', 'forward'][predicted_class_index]

    print("Predicted class:", predicted_class)

Enter the image number ('q' to quit): 4
1/1 [==============================] - 2s 2s/step
Predicted class: forward
Enter the image number ('q' to quit): 5
1/1 [==============================] - 0s 226ms/step
Predicted class: forward
Enter the image number ('q' to quit): 13
1/1 [==============================] - 0s 206ms/step
Predicted class: forward
Enter the image number ('q' to quit): q
